# Python file to Docker, to ECR, to Lambda 

## Manually Setup Docker Directory

lamdba_fxn_name/
├── Dockerfile
├── app/
│   ├── main.py          # Your main Lambda handler
│   ├── helper.py        # Your helper functions
│   └── requirements.txt # Python dependencies

## Custom Variables

In [1]:
lamdba_fxn_name = "nfl_all_games"

In [2]:
# List of AWS managed policy ARNs
managed_policies = [
    "arn:aws:iam::aws:policy/AmazonAPIGatewayAdministrator",
    "arn:aws:iam::aws:policy/AmazonAPIGatewayInvokeFullAccess",
    "arn:aws:iam::aws:policy/service-role/AmazonAPIGatewayPushToCloudWatchLogs",
    "arn:aws:iam::aws:policy/AmazonAthenaFullAccess",
    "arn:aws:iam::aws:policy/AmazonDynamoDBFullAccess",
    "arn:aws:iam::aws:policy/AmazonEC2ContainerRegistryFullAccess",
    "arn:aws:iam::aws:policy/AmazonS3FullAccess",
    "arn:aws:iam::aws:policy/AWSLambda_FullAccess",
]

## Dependencies

In [3]:
from packages.helpers.helpers import joel_boto

## Functions

In [4]:
def get_dependent_variables(lamdba_fxn_name):
    # Naviagte to the directory i want to build in (same name as lambda)
    path_docker = f"../docker/{lamdba_fxn_name}"

    # Get full IAM role ARN from the name
    role = jb.iam.get_role(RoleName=lamdba_fxn_name)
    role_arn = role["Role"]["Arn"]

    # Used to connect lamba to the correct ECR repo
    repo_uri = jb.get_ecr_repo_uri(lamdba_fxn_name)
    repo_image_uri = repo_uri + ':latest'


    return path_docker, role_arn, repo_uri, repo_image_uri

## Dependent Variables

In [5]:
# Connect to custom AWS class
jb = joel_boto()

running local credentials
✅ Logged in to ECR successfully.
✅ Connected to all clients successfully.


In [6]:
jb.create_lambda_iam_role(lamdba_fxn_name, managed_policies)

⚠️ IAM Role 'nfl_all_games' already exists.


In [7]:
# Create ECR Repository (one for each lambda fxn)
jb.create_ecr_repository(lamdba_fxn_name)

Repository 'nfl_all_games' already exists.


In [8]:
# Grab Dependent variables
path_docker, role_arn, repo_uri, repo_image_uri = get_dependent_variables(lamdba_fxn_name)

In [9]:
import os
print("DOCKER_HOST:", os.environ.get("DOCKER_HOST"))
print("DOCKER_API_VERSION:", os.environ.get("DOCKER_API_VERSION"))
print("DOCKER_CERT_PATH:", os.environ.get("DOCKER_CERT_PATH"))
print("DOCKER_TLS_VERIFY:", os.environ.get("DOCKER_TLS_VERIFY"))

DOCKER_HOST: None
DOCKER_API_VERSION: None
DOCKER_CERT_PATH: None
DOCKER_TLS_VERIFY: None


## Build, Push, and Create Lambda

In [10]:
# Build Docker image and push to ECR
jb.build_and_push_to_ECR(path_docker, repo_uri)

# Create a Lambda fxn using Image in ECR
jb.create_lambda_function(lamdba_fxn_name, role_arn, repo_image_uri)

DockerException: Error while fetching server API version: Not supported URL scheme http+docker

## Update Lambda if exists to the latest image in ECR

In [11]:
# Update if already Exists
jb.update_lambda_function(lamdba_fxn_name, repo_image_uri)

✅ Lambda updated: 2025-06-14T13:25:55.000+0000


## API Gateway

In [9]:
api_name = 'chalkjuice_api'
route_selection_expr = '$request.body.action'

In [10]:
lambda_client = jb.lambda_client
apigw = jb.apigw

In [17]:
jb.region

'us-east-2'

## Create API

In [14]:
# STEP 1: Create the WebSocket API
api = apigw.create_api(
    Name=api_name,
    ProtocolType='WEBSOCKET',
    RouteSelectionExpression=route_selection_expr
)
api_id = api['ApiId']
print(f"Created WebSocket API: {api_id}")

Created WebSocket API: 7aqddsnx56


In [ ]:
#api_id = '7aqddsnx56'

## Create Routes

In [23]:
paginator = lambda_client.get_paginator('list_functions')

print("Listing Lambda functions and ARNs:")

for page in paginator.paginate():
    for function in page['Functions']:
        print(f"Name: {function['FunctionName']}, ARN: {function['FunctionArn']}")

Listing Lambda functions and ARNs:
Name: api1_disconnect, ARN: arn:aws:lambda:us-east-2:026090519913:function:api1_disconnect
Name: sleeper, ARN: arn:aws:lambda:us-east-2:026090519913:function:sleeper
Name: api1_connect, ARN: arn:aws:lambda:us-east-2:026090519913:function:api1_connect
Name: nfl_all_games, ARN: arn:aws:lambda:us-east-2:026090519913:function:nfl_all_games
Name: nfl_matchups_model, ARN: arn:aws:lambda:us-east-2:026090519913:function:nfl_matchups_model


In [29]:
# Replace with your real Lambda ARNs
lambda_routes = {
    '$connect':'arn:aws:lambda:us-east-2:026090519913:function:api1_connect',
    '$disconnect':'arn:aws:lambda:us-east-2:026090519913:function:api1_disconnect',
    'nfl_matchups_model': 'arn:aws:lambda:us-east-2:026090519913:function:nfl_matchups_model',
    'nfl_all_games': 'arn:aws:lambda:us-east-2:026090519913:function:nfl_all_games'
}

In [27]:
# STEP 2: Create routes and integrations
for route_key, lambda_arn in lambda_routes.items():
    # Create integration
    integration = apigw.create_integration(
        ApiId=api_id,
        IntegrationType='AWS_PROXY',
        IntegrationUri=f'arn:aws:apigateway:us-east-2:lambda:path/2015-03-31/functions/{lambda_arn}/invocations'
    )

    integration_id = integration['IntegrationId']
    
    # Create route
    apigw.create_route(
        ApiId=api_id,
        RouteKey=route_key,
        Target=f'integrations/{integration_id}'
    )
    print(f"Connected route '{route_key}' to Lambda")

Connected route '$connect' to Lambda
Connected route '$disconnect' to Lambda
Connected route 'nfl_matchups_model' to Lambda
Connected route 'nfl_all_games' to Lambda


In [35]:
# Step 5: Add Lambda permissions for API Gateway to invoke the Lambdas
for lambda_arn in lambda_routes.values():
    fn_name = lambda_arn.split(':')[-1]
    statement_id = f'{api_name}-{fn_name}-invoke'
    try:
        lambda_client.add_permission(
            FunctionName=fn_name,
            StatementId=statement_id,
            Action='lambda:InvokeFunction',
            Principal='apigateway.amazonaws.com',
            SourceArn='arn:aws:execute-api:us-east-2:026090519913:7aqddsnx56/*'
        )
        print(f"Added permission for Lambda function {fn_name} to be invoked by API Gateway.")
    except lambda_client.exceptions.ResourceConflictException:
        print(f"Permission {statement_id} already exists, skipping.")

Permission chalkjuice_api-api1_connect-invoke already exists, skipping.
Added permission for Lambda function api1_disconnect to be invoked by API Gateway.
Added permission for Lambda function nfl_matchups_model to be invoked by API Gateway.
Added permission for Lambda function nfl_all_games to be invoked by API Gateway.


## MAKE SURE to manually add lambda triggers

#### Create Deployment

In [ ]:
stage = apigw.create_deployment(ApiId=api_id)

ConflictException: An error occurred (ConflictException) when calling the CreateStage operation: Stage already exists

#### Create Stage

In [ ]:
apigw.create_stage(
    ApiId=api_id,
    StageName='prod',
    DeploymentId=stage['DeploymentId']
)
print(f"Deployed API to 'prod' stage")

#### Update Stage

In [ ]:
deployment_id = stage['DeploymentId']
stage_name = 'prod'

apigw.update_stage(
    ApiId=api_id,
    StageName=stage_name,
    DeploymentId=deployment_id
)

{'ResponseMetadata': {'RequestId': '76830ab0-c6eb-4224-8f30-9b98e4fcb54b',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Sat, 14 Jun 2025 12:14:00 GMT',
   'content-type': 'application/json',
   'content-length': '342',
   'connection': 'keep-alive',
   'x-amzn-requestid': '76830ab0-c6eb-4224-8f30-9b98e4fcb54b',
   'access-control-allow-origin': '*',
   'x-amz-apigw-id': 'MJ0dXE8siYcEBEw=',
   'access-control-expose-headers': 'x-amzn-RequestId,x-amzn-ErrorType,x-amzn-ErrorMessage,Date',
   'x-amzn-trace-id': 'Root=1-684d6788-48a36d285be593056be612c2'},
  'RetryAttempts': 0},
 'CreatedDate': datetime.datetime(2025, 6, 14, 11, 12, 11, tzinfo=tzutc()),
 'DefaultRouteSettings': {'DataTraceEnabled': True,
  'DetailedMetricsEnabled': False,
  'LoggingLevel': 'ERROR'},
 'DeploymentId': 'ezacpq',
 'LastUpdatedDate': datetime.datetime(2025, 6, 14, 12, 14, tzinfo=tzutc()),
 'RouteSettings': {},
 'StageName': 'prod',
 'StageVariables': {},
 'Tags': {}}

# test

In [28]:
response = lambda_client.get_policy(FunctionName='nfl_matchups_model')
print(response['Policy'])

ResourceNotFoundException: An error occurred (ResourceNotFoundException) when calling the GetPolicy operation: The resource you requested does not exist.